In [1]:
import sqlite3
from contextlib import closing

import pandas as pd

DB_PATH = 'databases/MMAuto[GF260112](12-01-26-090617).mmbak'

# with sqlite.connect() -> auto-closes transaction but DOES NOT auto-close connection
# that's why also called contextlib.connect()
# https://blog.rtwilson.com/a-python-sqlite3-context-manager-gotcha/
# with closing(sqlite3.connect(DB_PATH)) as connection:   
#     pass  

connection = sqlite3.connect(DB_PATH)
# cursor = connection.cursor()

In [ ]:
query = """
select datetime(t.UTIME / 1000, 'unixepoch') as dt,
       category.NAME as category,
       c.SYMBOL || ' ' || t.AMOUNT_ACCOUNT as amount,
       fromAssets.NIC_NAME as fromAsset,
       toAssets.NIC_NAME as toAsset,
       t.ZCONTENT as name, 
       t.ZDATA as description
from INOUTCOME t
join ZCATEGORY category on category.uid = t.ctgUid
join ASSETS fromAssets on fromAssets.uid = t.assetUid
left join ASSETS toAssets on toAssets.uid = t.toAssetUid
join CURRENCY c on c.uid = t.currencyUid
"""
transactions = pd.read_sql(query, connection)
transactions

,dt,category,amount,fromAsset,toAsset,name,description
0,2022-05-23 15:02:06,Salary,₹ 17000.0,ICICI Salary Account,None,,
1,2022-05-24 01:51:24,Transportation,₹ 600.0,Metro Card,None,Weekly home to office and back cost,
2,2022-05-25 04:25:38,Transportation,₹ 20.0,Cash,None,Auto,Metro to Office
3,2022-05-25 05:53:21,Food,₹ 10.0,UCO Savings Account,None,Hide & Seek,From Smart Store (at work)
4,2022-07-13 03:46:34,Salary,₹ 8246.0,Employee PF,None,Mfilterit Salary (EPF Deduction),Automatically deducted from salary each month ...
...,...,...,...,...,...,...,...
2063,2026-01-11 04:19:34,Health,₹ 5.0,Cash,None,Saran ashram dentist filling removal,
2064,2026-01-11 11:09:32,Snacks & Beverages,₹ 5.0,Cash,None,sweets,
2065,2026-01-11 11:25:28,Kitchen,₹ 150.0,Cash,None,Khajoor,
2066,2026-01-11 11:32:28,Kitchen,₹ 65.0,Cash,None,"Bananas, Guava",


In [7]:
query = """
select budgetUid, AMOUNT
from BUDGET_AMOUNT
where IS_DEL = 0
"""
budget = pd.read_sql(query, connection)
budget

,budgetUid,AMOUNT
0,9_1_0_0_6,1500.0
1,6_1_0_0_6,0.0
2,11_1_0_0_6,200.0
3,12_1_0_0_6,100.0
4,14_1_0_0_6,500.0
5,17_1_0_0_6,1000.0
6,17_1_0_0_6,1000.0
7,17_1_0_0_6,1000.0
8,6_1_0_0_6,1000.0
9,6_1_0_0_6,1000.0


In [23]:
query = """
select g.ACC_GROUP_NAME as accountGroup,
       t.NIC_NAME as name,
       t.ZDATA1 as description
from ASSETS t
join ASSETGROUP g on g.uid = t.groupUid
where t.ZDATA = 0
order by t.ORDERSEQ
"""
assets = pd.read_sql(query, connection)
assets

,accountGroup,name,description
0,Emergency Fund,Emergency Fund,Goal: 10L for car or flat\n-------- OLD ------...
1,Savings,Cash,"3000 in bag, rest in wallet"
2,Savings,UCO Savings Account,09500110036105\nUPI: sohangchopra-1@okhdfcbank...
3,Savings,ICICI Salary Account,008701053026\nUPI: sohangchopra-1@okicici (GPa...
4,Top-Up/Prepaid,Metro Card,
5,Investments,Quant Small Cap Fund,
6,Investments,Zerodha Kite Stocks,Discount Broker\nNeed to use instead of ICICI ...
7,Retirement,ICICI PPF,000418476401\nMatures in 15 years (2037)
8,Retirement,Employee PF,DSNHP16818210000010262\nMonthly:\nEmployee Sha...


In [18]:
query = """
select category.NAME as category,
       fromAssets.NIC_NAME as fromAsset,
       toAssets.NIC_NAME as toAsset,
       c.SYMBOL || ' ' || t.AMOUNT_SUB as amount,
       t.PAYEE as name,
       t.MEMO as description
from REPEATTRANSACTION t
left join ZCATEGORY category on category.uid = t.ctgUid
join ASSETS fromAssets on fromAssets.uid = t.assetUid
left join ASSETS toAssets on toAssets.uid = t.toAssetUid
join CURRENCY c on c.uid = t.currencyUid
where t.IS_DEL = 0
"""
repeat_transactions = pd.read_sql(query, connection)
repeat_transactions

,category,fromAsset,toAsset,amount,name,description
0,Salary,Employee PF,None,₹ 5000.0,PF by HCL (Sankalp Semiconductor Ltd),
1,None,ICICI PPF,ICICI Salary Account,₹ 500.0,PPF,Minimum Contribution
2,Donation,UCO Savings Account,None,₹ 3000.0,Newslaundry Annual Subscription,AutoPay is setup via GPay till March 2033\nCan...
3,Donation,ICICI Salary Account,None,₹ 3090.0,Educate Girls NGO,"Autopay setup in GPay UPI, \nCan cancel from t..."
4,Other,ICICI Salary Account,None,₹ 3000.0,Boring Money Annual Subscription,
5,Donation,ICICI Salary Account,None,₹ 119.0,Kurzgesagt: Paid Membership: youtube,Via GPay monthly autopay
6,Donation,ICICI Salary Account,None,₹ 2950.0,To Scroll.in news,via GPay AutoPay
7,Donation,ICICI Salary Account,None,₹ 59.0,Sarthak Goswami Youtube channel subscription,


In [21]:
query = """
select memoDate, title, content
from MEMO
where isDel = 0
order by orderSeq
"""
notes = pd.read_sql(query, connection)
notes

,memoDate,title,content
0,2023-06-28,Noida Stay126 PG,"Mom paid Rs. 32,500 (2 months advance, 1 month..."
1,2023-11-06,56k: Navi -> Quant,\n
2,2023-11-05,,20k RD transfer got missed due to less funds i...
3,2023-12-25,Chyawanprash Seva,Monday-Saturday: 25-30 December\n9 a.m. - ~4:1...
4,2024-02-11,Went to Parakh Wedding Satsang,\n
5,2024-04-07,"Phone Damaged, repl with Dad phone next Agra trip",\n
6,2024-06-23,Met Rishang @ Bhagat Halwai,\n
7,2024-06-28,Changed Toothbrush,\n
8,2024-08-06,IRCTC Ticket Refund,"Vande Bharat : received Rs. 1.5K, charge: Rs. ..."
9,2024-10-05,Left Noida PG,Monu (Stay126 PG owner) didn't refund Rs. 15K ...
